<a href="https://colab.research.google.com/github/fabienMoutarde/DLcourse/blob/master/Practical_visualObjectDetection_YOLOv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visual Objects Detection with YOLO_v3

YOLO_v3 is currently among the fastest Convolutional Neural Networks for visual objects detection. [To get more information about YOLO_v3, you should go to the official YOLO page of its creator Joseph Redmon: https://pjreddie.com/darknet/yolo/](https://pjreddie.com/darknet/yolo/). Note that the corresponding original implementation is written **in C language** within the [Darknet Open Source Neural Networks C library](https://pjreddie.com/darknet/)

This notebook was adapted by Pr Fabien Moutarde from https://github.com/tugstugi/dl-colab-notebooks/blob/master/notebooks/YOLOv3_PyTorch.ipynb, which uses a [PyTorch port of YOLO_v3 by Ayoosh Kathuria](https://github.com/ayooshkathuria/pytorch-yolo-v3) to detect objects on a given image. Note that this PyTorch implementation of YOLO_v3 is probably slower than the original version compiled from C.


## Configuration

*Make sure the python package for **git** and **wget** are installed*, otherwise do it. If using conda, open (possibly as Administrator) an "Anaconda Power Shell prompt", and execute **conda install git** into it, then execute **pip install wget**.

Also *make sure that the python package for **torch** is installed*, and otherwise install it.
If using conda, open (possibly as Administrator) an "Anaconda Power Shell prompt", and execute **conda install pytorch** into it.

Finally, *make sure that the python package for **opencv** is installed*, and otherwise install it.
If using conda, open (possibly as Administrator) an "Anaconda Power Shell prompt", and execute **conda install opencv** into it.

In [0]:
import sys
print('Your python version: {}'.format(sys.version_info.major))
# Uncomment lines below only if you need them:
#  On Windows/Anaconda
#!{sys.executable} -m pip install wget
#!{sys.executable} -m conda install git opencv keras
#!{sys.executable} -m conda install pytorch torchvision cudatoolkit=10.0 -c pytorch
#
#  WITHOUT Conda
#!{sys.executable} -m pip install -U git wget opencv keras --user
#!{sys.executable} -m pip install -U pytorch torchvision --user # TO BE ADAPTED DEPENDING ON Target&CUDA level


In [0]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/ayooshkathuria/pytorch-yolo-v3.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone  $git_repo_url
  #!cd $project_name && pip install -q -r requirement.txt
  
import sys
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

### Check if pyTorch is accessing GPU

In [0]:
import torch
cudaOK = torch.cuda.is_available()
if cudaOK:
    print("CUDA available from PyTorch == TRUE")
    print(torch.cuda.get_device_properties(0))


## Download official YOLO_v3 pretrained weights

In [0]:
#import wget
if not exists('yolov3.weights'):
  !wget -q https://pjreddie.com/media/files/yolov3.weights
  #filename = wget.download("https://pjreddie.com/media/files/yolov3.weights")
    

## Detect objects on a test image

First, download a standard test image from internet:

In [0]:
IMAGE_URL = 'https://raw.githubusercontent.com/tugstugi/dl-colab-notebooks/master/resources/dog.jpg'
image_file = basename(IMAGE_URL)
!wget -q -O $image_file $IMAGE_URL
!ls
image_name = basename(IMAGE_URL)

#image_file = wget.download('https://raw.githubusercontent.com/tugstugi/dl-colab-notebooks/master/resources/dog.jpg')
#plt.imshow(matplotlib.image.imread(image_file))
#image_name = 'dog.jpg' 

plt.imshow(matplotlib.image.imread(image_name))

Execute `detect.py` on that image and show the result:

In [0]:
!cd pytorch-yolo-v3 && python detect.py --weights ../yolov3.weights --images ../$image_name --det ..

plt.figure(figsize=(15, 10))
plt.imshow(matplotlib.image.imread('det_%s' % image_name))

### Now, try it on a real images from a camera on-board a car

In [0]:
# Uncomment below ONLY IF RUNNING ON Colab, and if you need access to your GoogleDrive
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"

In [0]:
# Copy whatever image you want to test on, and display it

image_file = 'example_front-imageREAL.png'

# Uncomment below ONLY IF RUNNING ON Colab, and if the test image is on your GoogleDrive
!cp "/content/drive/My Drive/"$image_file "."
!ls

# OTHERWISE, the Image file is supposed to simply be in same directory as the notebook
#
plt.imshow(matplotlib.image.imread(image_file))

In [0]:
# Apply YOLO_v3 on image_file, and display the visual objects detection result
#
!cd pytorch-yolo-v3 && python detect.py --weights ../yolov3.weights --confidence 0.5 --images ../$image_file --det ..

plt.figure(figsize=(15, 10))
plt.title('With confidence_threshold=0.5')
plt.imshow(matplotlib.image.imread('det_%s' % image_file))

# Same with very low confidence threshold (--> better sensitivity, but lower specificity)
#
!cd pytorch-yolo-v3 && python detect.py --weights ../yolov3.weights --confidence 0.1 --images ../$image_file --det ..

plt.figure(figsize=(15, 10))
plt.title('With confidence_threshold=0.1')
plt.imshow(matplotlib.image.imread('det_%s' % image_file))

# Same with very high confidence threshold (--> better specificity but lower sensitivity)
#
!cd pytorch-yolo-v3 && python detect.py --weights ../yolov3.weights --confidence 0.9 --images ../$image_file --det ..

plt.figure(figsize=(15, 10))
plt.title('With confidence_threshold=0.9')
plt.imshow(matplotlib.image.imread('det_%s' % image_file))



#### Now, check experimentally the influence of the confidence threshold, and of other detection parameters: NMS Threshhold, input resolution, scales (check out the source code of detect.py to find how they can be modified just by just adding or changing arguments used) 

### Same test on an image from CARLA driving simulator


In [0]:
image_file = 'example_front-imageCARLA.png'

# Uncomment below ONLY IF RUNNING ON Colab, and if the test image is on your GoogleDrive
!cp "/content/drive/My Drive/"$image_file "."
!ls

# OTHERWISE, the Image file is supposed to simply be in same directory as the notebook
#

plt.imshow(matplotlib.image.imread(image_file))


In [0]:
# Apply YOLO_v3 on image_file, and display the visual objects detection result
#
!cd pytorch-yolo-v3 && python detect.py --weights ../yolov3.weights --confidence 0.5 --images ../$image_file --det ..

plt.figure(figsize=(15, 10))
plt.imshow(matplotlib.image.imread('det_%s' % image_file))

#### Now, check experimentally the influence of the confidence threshold, and of other detection parameters: NMS Threshhold, input resolution, scales (check out the source code of detect.py to find how they can be modified just by just adding or changing arguments used) 

####**[Optional] Train/fine-tune YOLOv3 for detection of other types of objects**